# Table of Contents
- [Installation](#Installation)
- [Setup](#Setup)
- [Run.py](#Run-py)
- [Argo UI](#Argo-UI)

# <a name="Installation"></a> Installation

In [ ]:
from dflow import (
    ShellOPTemplate,
    InputParameter,
    OutputArtifact,
)
templ = ShellOPTemplate(image="alpine",
                        script="cat {{inputs.artifacts.inp}} &&\
                                echo {{inputs.parameters.msg}}\
                                > msg.txt",
                        )
templ.inputs.parameters["msg"] = InputParameter()
templ.inputs.artifacts["inp"] = InputArtifact(path="/tmp")
templ.outputs.artifacts["out"] = OutputArtifact(path="/tmp/msg.txt")

from dflow import Step
step1 = Step(name='Step1',template=templ,
            parameters={
                "param": "Hello"
            },
            artifacts={
                "inp": step0.outputs.artifacts["out"] 
            }
            )


## Prerequisite 

**(Skip this step if already installed)**

Before using [dflow](#https://github.com/deepmodeling/dflow), we need to install the following two things:
- Docker (Official installation instruction: https://docs.docker.com/desktop/mac/install/)
- Minikube (Official installation instruction: https://minikube.sigs.k8s.io/docs/start/)

## Install pydflow

In [1]:
!pip install pydflow

     |████████████████████████████████| 55 kB 110 kB/s eta 0:00:01
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached argo_workflows-5.0.0-py3-none-any.whl (452 kB)
  Using cached jsonpickle-2.2.0-py2.py3-none-any.whl (39 kB)
     |████████████████████████████████| 197 kB 259 kB/s eta 0:00:01
  Using cached kubernetes-23.6.0-py2.py3-none-any.whl (1.5 MB)
  Using cached minio-7.1.9-py3-none-any.whl (76 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Using cached requests-2.28.0-py3-none-any.whl (62 kB)
  Using cached websocket_client-1.3.2-py3-none-any.whl (54 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_auth-2.7.0-py2.py3-none-any.whl (160 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached idna-3.3-py3-non

**Once installed, restart the jupyter notebook kernel to make the installation to take effect.**

# <a name="Setup"></a> Setup

## Minikube

Dflow runs on kubernetes (k8s), so we need to start minikube

In [4]:
!minikube start --memory 2048 --cpus 2

😄  minikube v1.25.2 on Darwin 12.3.1 (arm64)
✨  Automatically selected the docker driver
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=2048MB) ...
🐳  Preparing Kubernetes v1.23.3 on Docker 20.10.12 ...
    ▪ kubelet.housekeeping-interval=5m
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Argo-workflows

Dflow is built on [argo-workflow](https://github.com/argoproj/argo-workflows), so we need to setup argo engine in k8s:

1. To get started quickly, we can use the quick start manifest which will install Argo Workflows as well as some commonly used components:

In [28]:
!kubectl create ns argo
!kubectl apply -n argo -f https://raw.githubusercontent.com/deepmodeling/dflow/master/manifests/quick-start-postgres.yaml

namespace/argo created
customresourcedefinition.apiextensions.k8s.io/clusterworkflowtemplates.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/cronworkflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workfloweventbindings.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtaskresults.argoproj.io configured
customresourcedefinition.apiextensions.k8s.io/workflowtasksets.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtemplates.argoproj.io unchanged
serviceaccount/argo created
serviceaccount/argo-server created
serviceaccount/github.com created
role.rbac.authorization.k8s.io/agent created
role.rbac.authorization.k8s.io/argo-role created
role.rbac.authorization.k8s.io/argo-server-role created
role.rbac.authorization.k8s.io/executor created
role.rbac.authorization.k8s.io/pod-manager created
role.rbac.authorization.k8s.

2. To monitor the setup progress, we can look at the pod status

In [32]:
!kubectl get pod -n argo

NAME                                  READY   STATUS    RESTARTS      AGE
argo-server-78f47df69f-7mvwm          0/1     Running   2 (37s ago)   56s
minio-5d6dccd444-f8mwb                1/1     Running   0             56s
postgres-869f7fbd7f-zzmw9             1/1     Running   0             56s
workflow-controller-586fdf9f6-fd52d   1/1     Running   1 (41s ago)   56s


**NOTE!!!!**: This process might take a while, depending on the internet speed. Wait and keep refreshing the above cell. Once the `STATUS` of all pods is `RUNNING`, you can proceed with the next step.

**IMPORTANT!!!!**

3. Open a port-forward so you can access the UI:

    Since we need to keep this UI running, we can run this command in the terminal:
    
```shell
kubectl -n argo port-forward deployment/argo-server 2746:2746 --address 0.0.0.0
```

We can access the Argo UI: https://127.0.0.1:2746

Security warning will be shown but we can safely ignore it. This is because we haven't add ceritificate to this address. 

4. Open a port-forward so you can access the artifact repository
    
    Open another terminal and run this, because you want to keep artifact respository running. Note that you don't need to ingress the artifact repository if you are not downloading or uploading artifact.
    
```shell
kubectl -n argo port-forward deployment/minio 9000:9000 --address 0.0.0.0
```

# <a name="Run-py"></a> Run.py

In the previous steps, we finished installing and seting up the necessary tools and environments for dflow to run. In this section, we will prepare a simple python script using dflow.

Imagine, we want to achieve the following workflow:

Step 1. 
1. Echo a string to msg.txt 
    
2. Echo a number and redirect it to results.txt 
    
Step 2.
1. Duplicate the content in msg.txt two times and redirect it to a new file
    
2. Get the value in results.txt and times the number by 2 and redirect it to results.txt

To construct a workflow in dflow, three parts are needed:
1. Construct OP templates
2. Instantiate the OP template to Step
3. Put steps together and submit the workflow

## Construct OP template

As explained in the [readme](https://github.com/dptech-corp/dflow#122--op-template), OP template is the fundamental component in dflow. For this particular workflow above, we need two OP templates:

For step 1:

In [33]:
from dflow import ShellOPTemplate
step1_templ = ShellOPTemplate(
                name="Hello",
                image="alpine:latest",
                script="echo {{inputs.parameters.msg}} > /tmp/msg.txt && echo {{inputs.parameters.number}} > /tmp/results.txt",
)

This defines the operation to be executed. Next, we need to setup the inputs and outputs for this step.

In [34]:
from dflow import InputParameter, OutputParameter, OutputArtifact
step1_templ.inputs.parameters = {
                            "msg": InputParameter(),
                            "number": InputParameter(),
}
step1_templ.outputs.parameters = {
                            "out_param": OutputParameter(value_from_path="/tmp/results.txt")
}
step1_templ.outputs.artifacts = {
                            "out_art": OutputArtifact(path="/tmp/msg.txt")
}

For step 2: 

In [35]:
step2_templ = ShellOPTemplate(
                name="Duplicate",
                image="alpine:latest",
                script="cat /tmp/foo.txt /tmp/foo.txt > /tmp/bar.txt && echo $(({{inputs.parameters.number}}*2)) > /tmp/results.txt",
)

This defines the operation to be executed. Notice 2 things:
1. We duplicated the content in `/tmp/foo.txt` 2 times, instead of `/tmp/msg.txt` in step 1. This is because OPTemplates are indepednent of each other. To make `/tmp/foo.txt` the same as `/tmp/msg.txt`, we only need to initialize it correctly when instantiating the OP template.
2. We redirected the output of the arithmetic operation to `/tmp/results.txt`. This is not the file appeared in step 1.

In [36]:
from dflow import InputArtifact
step2_templ.inputs.artifacts = {
                            "in_art":InputArtifact(path="/tmp/foo.txt") 
}
step2_templ.inputs.parameters = {
                            "number": InputParameter(),
}
step2_templ.outputs.artifacts = {
                            "out_art": OutputArtifact(path="/tmp/bar.txt")
}
step2_templ.outputs.parameters = {
                            "out_param": OutputParameter(value_from_path="/tmp/results.txt")
}

## Instantiate the OP template to Step

`Step` in the central block for building a workflow. A `Step` is created by instantiating an OP template. When a `Step` is initialized, values of all input parameters and sources of all input artifacts declared in the OP template must be specified.

In [16]:
from dflow import Step

step1 = Step (
            name="step1",
            template=step1_templ,
            parameters={"msg":"HelloWorld!", "number": 1},
)
step2 = Step(
            name="step2",
            template=step2_templ,
            parameters={"number":step1.outputs.parameters["out_param"]},
            artifacts={"in_art":step1.outputs.artifacts["out_art"]},
)

Step 1 takes in two values as parameters: "HelloWorld!" and 1. Step 2 takes the values and files from step 1 as the input parameters and artifacts.

## Put steps together and submit a workflow

We finished building blocks of this workflow. Now we can put them togther

In [37]:
from dflow import Workflow
wf = Workflow(name="helloworld")
wf.add(step1)
wf.add(step2)

This creates a workflow with name "helloworld" and adds two steps in series.

We can then submit the workflow. One caveat: we will get warning about certificiate verification since we haven't yet added cerificate to the address we specified for the UI. To suppress it, we can run the following 

In [38]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [39]:
wf.submit()

Workflow has been submitted (ID: helloworld-kjhkr)


{'metadata': {'name': 'helloworld-kjhkr', 'generateName': 'helloworld-', 'namespace': 'argo', 'uid': '7dfcdfaa-7b51-4ef9-847d-e746679a01f1', 'resourceVersion': '20896', 'generation': 1, 'creationTimestamp': '2022-06-14T10:57:17Z', 'labels': {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2022-06-14T10:57:17Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}}, 'f:spec': {}, 'f:status': {}}}]}, 'spec': {'templates': [{'name': 'helloworld-steps', 'inputs': {}, 'outputs': {}, 'metadata': {}, 'steps': [[{'name': 'step1', 'template': 'hello', 'arguments': {'parameters': [{'name': 'msg', 'value': 'HelloWorld!'}, {'name': 'number', 'value': '1'}]}, 'continueOn': {}}], [{'name': 'step2', 'template': 'duplicate', 'arguments': {'parameters': [{'name': 'number', 'value': "{{=

Another caveat: if you want to rerun the workflow, you need to start a new workflow by reruning `wf = Workflow(name="helloworld")`

# <a name="Argo-UI"></a> Argo UI

After finishing the previous steps, we can access the workflow we just ran on the UI (https://127.0.0.1:2746)

We should see the following once loaded.

<img src="./imgs/argoui_main_page.png" alt="argoUI_mainpage"/>

We can see the workflow we just ran. Left click it then we can see the following.

<img src="./imgs/workflow_overview.png" alt="workflow_overview"/>

This gives us an overview of the workflow. The first node does not do anything. The two following nodes are the steps specified by us. Click on it then we can see more information about each step.

We can access the input/outputs of step 2. We can see the parameters of the step on the UI. We can download `out_art` by clicking the download buttom. 

<img src="./imgs/access_one_node.png" alt="access_one_node"/>

After decompressing it, you should see a file named `bar.txt`. (This is also what we specified). Open it, you should see `HelloWorld!\nHelloWorld!`